# Extract from non-interlaced pages from Lotsawa

In [11]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from util import *

data = pd.read_csv("bisent_lotsawa_interlace.csv")
urls = pd.read_csv("lotsawa_urls.csv").sample(frac=1, random_state=4)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
urls_interlace = set(data.url.values)
urls_noninterlace = [url for url in urls.url if url not in urls_interlace]

In [13]:
len(urls_noninterlace)
urls_noninterlace[:2]

['https://www.lotsawahouse.org/tibetan-masters/khenpo-shenga/advice-on-yidam',
 'https://www.lotsawahouse.org/tibetan-masters/alak-zenkar/brief-biography-of-jamyang-khyentse-wangpo']

In [15]:
for url in urls_noninterlace[:]:
#     save_bilingual_tokenized(url)
    save_bilingual_plain(url)


Processing:  https://www.lotsawahouse.org/tibetan-masters/khenpo-shenga/advice-on-yidam   https://www.lotsawahouse.org/bo/tibetan-masters/khenpo-shenga/advice-on-yidam
Processing:  https://www.lotsawahouse.org/tibetan-masters/alak-zenkar/brief-biography-of-jamyang-khyentse-wangpo   https://www.lotsawahouse.org/bo/tibetan-masters/alak-zenkar/brief-biography-of-jamyang-khyentse-wangpo


KeyboardInterrupt: 

In [16]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

num_cores = multiprocessing.cpu_count()

processed_list = Parallel(n_jobs=num_cores)(delayed(save_bilingual_plain)(url) for url in tqdm(urls_noninterlace))


100%|██████████| 471/471 [02:43<00:00,  2.87it/s]


# Development code below

In [37]:
import requests
from bs4 import BeautifulSoup
import html2text
h = html2text.HTML2Text()
h.body_width = 0
h.ul_item_mark = ''
h.emphasis_mark = ''
h.strong_mark = ''

# urls_noninterlace[:10]
url = 'https://www.lotsawahouse.org/tibetan-masters/khenpo-shenga/candrakirti-praise'
bo_url = get_bo_url(url)
print(url, " ", bo_url)

r = requests.get(bo_url)
soup = BeautifulSoup(r.content, 'html.parser')
bo_text = soup.select("#maintext")[0].text

r = requests.get(url)
             
soup = BeautifulSoup(str(r.content, "utf-8").replace("\n", " "), 'html.parser')
for div in soup.find_all(["div", 'a'], {'class':['footnotes', 'footnote']}): 
    div.decompose()
for div in soup.find_all(["span"], {'class':['TibetanInlineEnglish']}): 
    div.decompose()
    
en_text = h.handle(str(soup.select("#maintext")[0]))

# import markdown
# html = markdown.markdown(en_text)
# en_text = "".join(BeautifulSoup(html).findAll(text=True))

https://www.lotsawahouse.org/tibetan-masters/khenpo-shenga/candrakirti-praise   https://www.lotsawahouse.org/bo/tibetan-masters/khenpo-shenga/candrakirti-praise



Although the results of our actions do not manifest immediately, they never go to waste. The Sūtra of a Hundred Actions says:

> The actions of beings never go to waste,  
>  Even in a hundred aeons.  
>  They are accumulated, and, once the time comes,  
>  The result will come to fruition.


In [45]:
bo_text.strip()
sents = []
for line in bo_text.strip().replace("\xa0", " ").split("\n"):
    if line:
        print(split_shay_pieces(line))

TypeError: 'tuple' object does not support item assignment

In [44]:
Chunks(line).make_chunks()


[(104, 0, 4),
 (104, 4, 2),
 (104, 6, 2),
 (104, 8, 2),
 (104, 10, 2),
 (104, 12, 2),
 (104, 14, 5)]

In [71]:
lines = get_lines_en(en_text)
lotsawa_en_line_filter(lines)
lines

['A Song of Advice for Gok Zangden',
 'Allowing inner awareness to be unrestricted,',
 'While remaining unattached to outer appearances:',
 'Beyond outer and inner, all is transparently clear—',
 'Free from hope and fear: how amazing!',
 'Distraction, when it occurs, is an aspect of ordinary mind,',
 'So, on such occasions, do not neglect cause and effect.',
 'When undistracted, let the wheel of awareness turn—',
 'May physical and mental restrictions be released!',
 'Through the power of inseparable space and awareness,',
 'Wherein whatever arises does not stray from absolute space,',
 'Fearlessness emerges out of the sky-like expanse:',
 'May you and I thus remain inseparable!',
 'Khenchen Gangshar Wangpo wrote this for Gok Zangden.']

In [72]:
[en_tokenize(s) for s in get_sents_en(en_text)]
# print(en_text)

['A Song of Advice for Gok Zangden',
 'Allowing inner awareness to be unrestricted ,',
 'While remaining unattached to outer appearances :',
 'Beyond outer and inner , all is transparently clear —',
 'Free from hope and fear : how amazing !',
 'Distraction , when it occurs , is an aspect of ordinary mind ,',
 'So , on such occasions , do not neglect cause and effect .',
 'When undistracted , let the wheel of awareness turn —',
 'May physical and mental restrictions be released !',
 'Through the power of inseparable space and awareness ,',
 'Wherein whatever arises does not stray from absolute space ,',
 'Fearlessness emerges out of the sky - like expanse :',
 'May you and I thus remain inseparable !',
 'Khenchen Gangshar Wangpo wrote this for Gok Zangden .']

In [35]:
[tokenize(s) for s in get_sents_bo(bo_text.strip())]

['གདམས་ངག་ ཚིག་ གཅིག་ མ་ ནི །',
 'བླ་མ་ ལ་ ཕྱག་ འཚལ་ ལོ །',
 'འདི ར་ རྫོགས་པ་ ཆེན་པོ འི་ སྐབས་ ཀྱི་ སེམས་ ངོ་ ཟེར་བ །',
 'སེམས་ ཀྱི་ གཤིས་ རང་བབས་ ཆོས་ཉིད་ ཀྱི་ ཐོག་ ཏུ་ མ་བཅོས་པ ར་ བཞག་པ་ སྟེ །',
 'མ་ ཡེངས་ སེམས་ ཀྱི་ རང་ གིས་ རང་ ལ་ བལྟ །',
 'ཞེས་ ཁ་ ནང་ ལོག་ གིས་ ཤེས་པ་ རང་གནས་ ཐ་མལ་ གྱི་ ཤེས་པ་ རྗེན་ ལ་ བུད་པ་ ཞིག་ འཆར་ རྒྱུ་ ཡོད་པ །',
 'ཟ་ཉལ་ འགྲོ་འདུག་ གི་ དུས་ ཐམས་ཅད་ ན་ མི་ བརྗེད་པ ར་ དྲན་པ ས་ རྒྱང་ འཚོ་བྱེད་ ནས་ མི་རྒན་ གྱིས་ བྱིས་པ འི་ རྩེད་མོ་ མཐོང་བ་ བཞིན །',
 'གྲོལ་ ས འི་ ཆོས་སྐུ་ ངོས་བཟུང་ ནས །',
 'རྩལ་ གྱི་ རྣམ་རྟོག་ ཤར་ གྲོལ་ རྗེས་མེད་ ལམ་ དུ་ འཁྱེར་ དགོས་ སོ །།',
 'ཞེས་ གདམས་ངག་ ཚིག་ གཅིག་ མ་ མཁན་ཆེན་ ངག་དབང་ དཔལ་ བཟང་ གིས་ གསུངས་ སོ །།']

In [73]:
en_text

'## A Song of Advice for Gok Zangden\n\nby Khenpo Gangshar\n\n  \n  \n  \n  \nAllowing inner awareness to be unrestricted,  \nWhile remaining unattached to outer appearances:  \nBeyond outer and inner, all is transparently clear—  \nFree from hope and fear: how amazing!\n\n  \n  \n  \n  \nDistraction, when it occurs, is an aspect of ordinary mind,  \nSo, on such occasions, do not neglect cause and effect.  \nWhen undistracted, let the wheel of awareness turn—  \nMay physical and mental restrictions be released!\n\n  \n  \n  \n  \nThrough the power of inseparable space and awareness,  \nWherein whatever arises does not stray from absolute space,  \nFearlessness emerges out of the sky-like expanse:  \nMay you and I thus remain inseparable!\n\nKhenchen Gangshar Wangpo wrote this for Gok Zangden.\n\n| Translated by [Adam Pearcey](/translators/adam-pearcey/), 2018.\n'

## TODOs:
Specail cases:
* DONE! Tibetan inline cases: https://www.lotsawahouse.org/tibetan-masters/khenpo-gangshar/song-of-advice-for-gok-zangden
* DONE! Remove "By..." and "| Translated by..." 
* DONE! remove footnotes class https://www.lotsawahouse.org/tibetan-masters/jamyang-khyentse-chokyi-lodro/rajgir-vulture-peak-nalanda-praise
* DONE! replace \xa0 in en and bo text

In [3]:
save_bilingual_tokenized(url)

NameError: name 'url' is not defined